In [1]:
from typing import List, Dict

class Recipe:
    def __init__(self, user_id: str, context: Dict, user_scores: Dict[str, float], product_catalog: Dict[str, Dict]):
        self.user_id = user_id
        self.context = context
        self.user_scores = user_scores
        self.product_catalog = product_catalog

    def get_recommendations(self, top_k: int = 10) -> List[str]:
        raise NotImplementedError("Subclasses must implement this method.")


In [2]:
class ChosenForYouRecipe(Recipe):
    def filter_by_category(self, skus: List[str]) -> List[str]:
        target_category = self.context.get("category")
        return [
            sku for sku in skus
            if self.product_catalog.get(sku, {}).get("category") == target_category
        ]

    def rank_skus(self, skus: List[str]) -> List[str]:
        return sorted(skus, key=lambda x: self.user_scores.get(x, 0), reverse=True)

    def get_recommendations(self, top_k: int = 10) -> List[str]:
        all_skus = list(self.user_scores.keys())
        filtered = self.filter_by_category(all_skus)
        ranked = self.rank_skus(filtered)
        return ranked[:top_k]


In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class SimilarItemsRecipe:
    def __init__(self, sku_id: str, product_embeddings: Dict[str, np.ndarray], product_catalog: Dict[str, Dict]):
        self.sku_id = sku_id
        self.product_embeddings = product_embeddings
        self.product_catalog = product_catalog

    def get_similar_items(self, top_k: int = 5) -> List[str]:
        target_vec = self.product_embeddings[self.sku_id].reshape(1, -1)
        target_cat = self.product_catalog[self.sku_id]["category"]

        similarities = {
            sku: cosine_similarity(target_vec, emb.reshape(1, -1))[0][0]
            for sku, emb in self.product_embeddings.items()
            if sku != self.sku_id and self.product_catalog[sku]["category"] == target_cat
        }

        ranked = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        return [sku for sku, _ in ranked[:top_k]]


In [4]:
# Sample input
context = {"category": "hair"}
user_scores = {
    "sku_101": 0.92,
    "sku_102": 0.85,
    "sku_103": 0.30,
    "sku_104": 0.65,
    "sku_105": 0.78
}
product_catalog = {
    "sku_101": {"category": "hair", "brand": "BrandA"},
    "sku_102": {"category": "makeup", "brand": "BrandB"},
    "sku_103": {"category": "hair", "brand": "BrandC"},
    "sku_104": {"category": "skin", "brand": "BrandD"},
    "sku_105": {"category": "hair", "brand": "BrandA"},
}

# Instantiate and run ChosenForYou
chosen = ChosenForYouRecipe(
    user_id="user_456",
    context=context,
    user_scores=user_scores,
    product_catalog=product_catalog
)
print("Chosen for You:", chosen.get_recommendations(top_k=3))


Chosen for You: ['sku_101', 'sku_105', 'sku_103']
